# MCQ (Multiple Choice Question) 평가 튜토리얼

## MCQDataset

이 튜토리얼에서는 Huggingface의 객관식 dataset을 불러와서 평가 후 재업로드하는 과정까지 경험해볼 것입니다.

### 1. 데이터셋 불러오기
먼저 HuggingFace Hub에서 데이터셋을 불러오는 방법을 알아보겠습니다:

In [1]:
from langmetrics.llmfactory import LLMFactory
from dotenv import load_dotenv
from langmetrics.config import ModelConfig
import os

In [2]:
load_dotenv(override=True)

True

In [3]:
LLMFactory.get_model_list()

['gpt-4o',
 'gpt-4o-mini',
 'deepseek-v3',
 'deepseek-reasoner',
 'claude-3.7-sonnet',
 'claude-3.5-sonnet',
 'claude-3.5-haiku',
 'naver',
 'gemini-2.0-flash']

In [7]:
deepseek_config =  ModelConfig(
            model_name="deepseek-chat",
            api_base="https://api.deepseek.com",
            api_key=os.getenv("DEEPSEEK_API_KEY"),
            max_tokens=500,
            seed=66,
            provider="deepseek"
        )

In [8]:
# localllm은 서버를 local에서 실행시키기 때문에 부팅되는 시간이 존재합니다.
custom_llm = LLMFactory.create_llm(deepseek_config, temperature=1.0)

ModelConfig(model_name='deepseek-chat', api_key='sk-73508341cbe64cc3916f480fd37e91f2', api_base='https://api.deepseek.com', seed=66, max_tokens=500, rpm=None, provider='deepseek')


한글 데이터셋 이므로 template을 한글 사용

In [25]:
from langmetrics.metrics import MCQMetric
metric = MCQMetric(
    answer_model=custom_llm,
    template_language='ko',  # 'ko' 또는 'en'
    generate_template_type='reasoning'  # 'reasoning' 또는 'only_answer'
)

async를 통해서 빠르게 추론을 할 것입니다.

In [16]:
import nest_asyncio
nest_asyncio.apply()

In [17]:
print(test_dataset[0])

LLMTestCase(input='2개월 남아가 BCG예방접종 1개월 뒤 주사 부위에 이상반응이 생겨서 예방접종을 실시한 소아청소년과의원을 찾아왔다. 이때 「감염병의 예방 및 관리에 관한 법률」에 따라 예방접종 후 이상반응으로 진단한 원장이 이상반응 발생신고서를 제출해야 할 대상은?', output=None, expected_output='C', context=None, retrieval_context=None, reasoning=None, choices=['대한의사협회장', '보건복지부장관', '남아 소재지 관할 보건소장', '남아 소재지 관할 시장 ∙ 군수 ∙ 구청장', '남아 소재지 관할 시 ∙ 도지사'])


In [18]:
results = await metric.ameasure(test_dataset, batch_size=512)

In [19]:
results

LLMDataset(Pandas DataFrame with 3009 rows)

In [20]:
# r1_results = await r1_metric.ameasure(test_dataset[:10])

약 1200개의 달하는 test를 단 30초만에 모두 추론한 것을 확인할 수 있습니다!

In [21]:
results.df['student_answer'][0]

'{\n"reasoning": "BCG 예방접종 후 이상반응으로 진단한 원장은 이진단을 환자에게 신고해야 하며, 이를 위해 의료기관의 소재지에 있는 보건소 또는 구청에 보고해야 합니다. 이진단은 심각도를 고려하여 \'주의 사항\', \'중요 사항\', 또는 \'요구 사항\' 중 하나를 선택하여 보고합니다. 남아의 소재지 관할 보건소장이 보고처이므로, 보건소 또는 구청장이 아니라 보건소장이 더 적합합니다. 또한, 보건소장의 성명은 보건복지부의 영역이므로 보건복지부장관의 보고처는 아닙니다. 따라서 남아 소재지 관할 보건소장이 더 적절합니다.",\n"answer": "C"\n}'

In [22]:
results.df

,input,student_answer,teacher_answer,expected_output,context,retrieval_context,reasoning,choices,score,metadata
0,2개월 남아가 BCG예방접종 1개월 뒤 주사 부위에 이상반응이 생겨서 예방접종을 실...,"{\n""reasoning"": ""BCG 예방접종 후 이상반응으로 진단한 원장은 이진단...",None,C,None,None,"BCG 예방접종 후 이상반응으로 진단한 원장은 이진단을 환자에게 신고해야 하며, 이...","[대한의사협회장, 보건복지부장관, 남아 소재지 관할 보건소장, 남아 소재지 관할 시...",1,"{'student_template_language': 'ko', 'student_m..."
1,‘A’ 종합병원은 폐암으로 진단받은 환자 ‘B’에게 방사선치료와 항암화학요법을 시행...,"{\n""reasoning"": ""방사선치료와 항암화학요법이 국민건강보험법을 통해 지원...",None,C,None,None,방사선치료와 항암화학요법이 국민건강보험법을 통해 지원되는 환자 치료항목으로 분류되어...,"[보건복지부, 국민건강보험공단, 건강보험심사평가원, 의료기관인증위원회, 한국의료분쟁...",0,"{'student_template_language': 'ko', 'student_m..."
2,의사 ‘A’는 군지역에서 개인 의원을 운영하고 있다. 자녀 교육의 목적으로 2개월 ...,"{\n""reasoning"": ""의사가 개인 의원을 운영하므로, 이는 개인의 사업으로...",None,A,None,None,"의사가 개인 의원을 운영하므로, 이는 개인의 사업으로 기간별로 휴업을 신고해야 할 ...","[군수에게 휴업 신고, 도지사에게 휴업 신고, 보건복지부장관에게 휴업 신고, 국민건...",0,"{'student_template_language': 'ko', 'student_m..."
3,종합병원에 근무하는 의사 ‘A’는 미국에서 개최되는 학회에 참석하기 위하여 7일간 ...,"{\n""reasoning"": ""이 문제에서 중요한 요소는 ‘A’ 의사의 부재기간 동...",None,C,None,None,이 문제에서 중요한 요소는 ‘A’ 의사의 부재기간 동안 환자 ‘B’가 진단서 요청을...,"[진료한 의사 ‘A’가 없어서 발급할 수 없다고 안내한다., 원무과장이 ‘A’에게 ...",1,"{'student_template_language': 'ko', 'student_m..."
4,의원을 운영 중인 원장 ‘A’는 허리통증에 대한 최신 치료법을 광고하고 있다. ‘A...,"{\n""reasoning"": ""원장 'A'가 자신의 허리통증 치료 기법에 대해 확신...",None,E,None,None,"원장 'A'가 자신의 허리통증 치료 기법에 대해 확신했지만, 치료법이 실제로 모든 ...","[조치 없음, 경고, 벌금 부과, 과태료 부과, 1년의 범위에서 의료업 정지]",0,"{'student_template_language': 'ko', 'student_m..."
...,...,...,...,...,...,...,...,...,...,...
3004,「응급의료에 관한 법률」상 2급 응급구조사의 업무가 아닌 것은?,"{\n""reasoning"": ""응급의료에 관한 법률에서는 2급 응급구조사의 업무를 ...",None,A,None,None,응급의료에 관한 법률에서는 2급 응급구조사의 업무를 다음과 같이 규정하고 있습니다:...,"[정맥로의 확보, 기본 심폐소생술, 구강내 이물질의 제거, 외부출혈의 지혈 및 창상...",0,"{'student_template_language': 'ko', 'student_m..."
3005,「보건의료기본법」상 국가와 지방자치단체가 평생국민건강관리체계를 구축하기 위하여 시행...,"{\n""reasoning"": ""「보건의료기본법」은 주로 국민의 건강 기간별 관리체계...",None,C,None,None,「보건의료기본법」은 주로 국민의 건강 기간별 관리체계를 목표로 하고 있다. 따라서 ...,"[정신 보건의료, 환경 보건의료, 정신 보건의료, 구강 보건의료, 학교 보건의료, ...",0,"{'student_template_language': 'ko', 'student_m..."
3006,"「국민건강증진법」상 주류제조의 면허를 받은 자는 ""과다한 음주는 건강에 해롭다.""라...","{\n""reasoning"": ""「국민건강증진법」에서는 주류제조의 면허를 받은 자에 ...",None,A,None,None,「국민건강증진법」에서는 주류제조의 면허를 받은 자에 대해 '과다한 음주는 건강에 해...,"[1도, 3도, 7도, 11도, 17도]",0,"{'student_template_language': 'ko', 'student_m..."
3007,"「혈액관리법」상 혈액관리업무를 할 수 있는 자 중, 채혈을 할 수 없는 자는?","{\n""reasoning"": ""이 문제에서는 각 경우에 대해 한대 한대의 정보를 분...",None,E,None,None,이 문제에서는 각 경우에 대해 한대 한대의 정보를 분석하여 채혈을 할 수 없는 자가...,"[의원, 치과의원, 한의원, 대한적십자사, 혈액제제 제조업자]",0,"{'student_template_language': 'ko', 'student_m..."


In [23]:
scores = sum([i.score for i in results]) / len(results)

In [24]:
print(scores)

0.3765370555001662
